In [ ]:
%run optimizers/Optimizer.ipynb

class PoseEstimation(Optimizer):
    
    def __init__(self, mean, pcaBasis, pcaVariance, numIterations=1000):
        Optimizer.__init__(self, mean, pcaBasis, pcaVariance)
        self.numIterations = numIterations
        self.a, self.r, self.t, self.s = Optimizer.getDefaults(self.numComponents)
    
    def open(self):
        self.targetFeatures = tf.placeholder(tf.float64, shape=(2*self.numFeatures, 1), name='targetFeatures')
        self._setVariables()
        self._setModelComponents()
        self._setModelFeatures()
        self._setAComponent()
        self._setError()
        self._setOptimizer()
        self.session = tf.Session()
        model = tf.global_variables_initializer()
        self.session.run(model)
    
    def close(self):
        self.session.close()
    
    def optimize(self, targetFeatures):
        print('RUNNING')
        for _ in range(self.numIterations):
            self.session.run(self.optimizer,
                    feed_dict={self.targetFeatures: np.reshape(targetFeatures, (2*self.numFeatures, 1))})
        print('DONE')
        k = np.array(self.session.run(self.k))
        
        r1 = np.array([k[0][0], k[1][0], k[2][0]])
        r2 = np.array([k[4][0], k[5][0], k[6][0]])

        s = (np.linalg.norm(r1) + np.linalg.norm(r2)) / 2.
        t = np.array([k[3][0]/s, k[7][0]/s]).reshape((2,1))

        U,S,V = np.linalg.svd(np.array([r1, r2, np.cross(r1, r2)]))
        r = np.matmul(U,V) # TODO, should .T??
        if np.linalg.det(r) == -1:
            U,S,V = np.linalg.svd(np.array([r1, r2, -1*np.cross(r1, r2)]))
            r = np.matmul(U,V.T) # TODO, should .T??
        
        return self.a, r, t, s
    
    def update(self, a, r, t, s):
        self.a = a
        self.r = r
        self.t = t
        self.s = s
        
    def _setVariables(self):
        self.k = tf.Variable(np.zeros((8,1), dtype=np.float64), name='k', trainable=True)
    
    def _setModelComponents(self):
        self.train_mean = tf.constant(self.mean, dtype=tf.float64)
        self.train_pcaBasis = tf.constant(self.pcaBasis, dtype=tf.float64)
        self.train_pcaVariance = tf.constant(self.pcaVariance, dtype=tf.float64)
    
    def _setModelFeatures(self):
        self.model_features = tf.add(self.train_mean, tf.matmul(self.train_pcaBasis, self.a))
    
    def _setAComponent(self):
        #session.run before optimize???
        Ai = tf.concat(
                values = [
                    tf.reshape(self.model_features, (self.numFeatures, 3)),
                    tf.ones([self.numFeatures, 1], tf.float64)
                ],
                axis = 1)
        Apad = tf.concat(
                values = [Ai, tf.zeros([self.numFeatures, 4], tf.float64)],
                axis = 1)
        padA = tf.concat(
                values = [tf.zeros([self.numFeatures, 4], tf.float64), Ai],
                axis = 1)
        self.A = tf.reshape(
                tensor = tf.concat(
                    values = [Apad, padA],
                    axis = 1
                ),
                shape = [2*self.numFeatures, 8])
    
    def _setError(self):
        expected = tf.transpose(tf.reshape(self.targetFeatures, (self.numFeatures, 2)))
        actual = tf.transpose(tf.reshape(tf.matmul(self.A, self.k), (self.numFeatures, 2)))
        self.error = tf.cast(tf.reduce_mean(tf.reduce_sum(tf.square(expected-actual), axis=0)), tf.float64)
    
    def _setOptimizer(self):
        self.optimizer = tf.train.AdamOptimizer(learning_rate=0.1).minimize(self.error)
    